# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [214]:
data = pd.read_csv('../data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [15]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [16]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [17]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [18]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [20]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [21]:
%%time

model = AlternatingLeastSquares(factors=125, 
                                regularization=0.01,
                                iterations=5, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 5.84 s, sys: 220 ms, total: 6.06 s
Wall time: 4.31 s


-----

# Домашнее задание

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
В prefilter нужно реализовать закомментированные ограничения.
4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их, как делалось на вебинаре
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

**Внимание!** модуль для вашего удобства - дополняйте его теми функциями, которые Вам пригодятся.  
Формальное задание только на prefiletr_items 

In [377]:
def prefilter_items(data_train, ids_by_departments, N_weeks=52, N_top=50, N_bot=50, N_top_cost=10, N_bot_cost=30):
    # Оставим только 5000 самых популярных товаров
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
    #добавим, чтобы не потерять юзеров
    data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999 
        
    # Уберем самые популярные 
    first = top_5000[:N_top] # выбираем топ 50 популярных товаров
    data_train = data_train.loc[~data_train['item_id'].isin(first)]
    
    # Уберем самые непопулряные 
    last = top_5000[len(top_5000) - N_bot:] # выбираем 50 не популярных товаров
    data_train = data_train.loc[~data_train['item_id'].isin(last)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    data_train = data_train[data_train.week_no > (data_train['week_no'].max() - N_weeks)]
    
    # Уберем не интересные для рекоммендаций категории (department)
    data_train = data_train.loc[~data_train['item_id'].isin(ids_by_departments)]
    
    # Расчеты стоимостей товаров
    quanti = data_train.groupby('item_id')['quantity'].sum().reset_index()
    cost = data_train.groupby('item_id')['sales_value'].sum().reset_index()
    cost['devide'] = cost.sales_value/quanti.quantity
    cost = cost[cost.devide != np.inf] # уберем inf
    cost = cost.fillna(0) # заполним NaN's
    
    # стоимость более 300 (долларов?)
    expensive = cost.loc[cost.devide > 300].sort_values('devide', ascending=False).item_id.to_list()
    # стоимость менеее 1 доллара
    less_1 = cost.loc[cost.devide > 1].sort_values('devide', ascending=False).item_id.to_list()
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    data_train = data_train.loc[~data['item_id'].isin(less_1)]
    
    # Уберем слишком дорогие товары
    data_train = data_train.loc[~data['item_id'].isin(expensive)]

    return data_train

def postfilter_items():
    pass

In [372]:
dz_data = data_train.copy()

In [373]:
dz_data.shape

(2278490, 12)

In [374]:
departments = ['GARDEN CENTER', 'RX', 'SALAD BAR', 'CNTRL/STORE SUP', 'AUTOMOTIVE']

In [375]:
departments_ids = item_features.loc[item_features['department'].isin(departments)].item_id.to_list()

In [378]:
dz_data = prefilter_items(dz_data, departments_ids)

In [379]:
dz_data.shape

(204079, 12)

In [381]:
dz_data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
816058,1774,31032664011,272,845262,2,1.38,356,-0.6,0,40,0.0,0.0
817508,1251,31046185485,272,976998,1,0.57,286,0.0,1531,40,0.0,0.0
817509,1163,31046185583,272,833723,1,0.30,286,0.0,1642,40,0.0,0.0
817521,1822,31046195732,272,1084591,1,0.50,31862,0.0,1229,40,0.0,0.0
817535,459,31046201115,272,5584027,2,0.78,31762,0.0,1550,40,0.0,0.0


In [105]:
def get_recommendations(user, model, sparse_user_item, N=5):
    """Рекомендуем топ-N товаров"""
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # !!! 
                                    recalculate_user=True)]
    return res

In [226]:
def get_similar_items_recommendation(data, user, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    populars = data.loc[data['user_id'] == user].sort_values('quantity', ascending=False) # найдем топ покупок юзера
    populars = populars[populars['item_id'] != 999999] 
    populars = populars['item_id'].head(N).to_list() # преобразуем для цикла
    
    res = []
    for popular in populars:
        recs = model.similar_items(itemid_to_id[popular], N=2)
        top_rec = recs[1][0]
        res.append(id_to_itemid[top_rec])
    
    return res

def get_similar_users_recommendation(user, model, user_item_matrix, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    sim_user = model.similar_users(userid_to_id[user], N=6)[1][0] # похожие юзеры
    
    own = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

    own.fit(csr_matrix(user_item_matrix).T.tocsr(), # топ для похожего юзера
          show_progress=False)
    
    res = get_recommendations(user=sim_user,
                        model=own,
                        sparse_user_item=csr_matrix(user_item_matrix).T.tocsr(), N=N)
    # your_code
    
    return res

In [133]:
get_similar_items_recommendation(data_train, 5, model)

[969568, 833025, 1007195, 5995503, 1058554]

In [134]:
get_similar_users_recommendation(5, model, user_item_matrix)

[963686, 936830, 854754, 870428, 931136]

##### Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py
##### get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их, как делалось на вебинаре

In [159]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

        user_item_matrix = user_item_matrix.astype(float) 
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model
    
    
    def get_similar_items_recommendation(self, data, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        populars = data.loc[data['user_id'] == user].sort_values('quantity', ascending=False) # найдем топ покупок юзера
        populars = populars[populars['item_id'] != 999999] 
        populars = populars['item_id'].head(N).to_list() # преобразуем для цикла
    
        res = []
        for popular in populars:
            recs = model.similar_items(itemid_to_id[popular], N=2)
            top_rec = recs[1][0]
            res.append(id_to_itemid[top_rec])

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
        
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        # your_code
        sim_user = model.similar_users(userid_to_id[user], N=6)[1][0] # похожие юзеры
    
        own = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

        own.fit(csr_matrix(user_item_matrix).T.tocsr(), # топ для похожего юзера
              show_progress=False)
    
        res = get_recommendations(user=sim_user,
                        model=own,
                        sparse_user_item=csr_matrix(user_item_matrix).T.tocsr(), N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

Проверка, что все работает

##### Почему не импортируется во втором варианте не могу разобраться

In [202]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
import src.recommenders # импортируется так

# Так не импортируется 
# from src.recommenders import MainRecommender 

In [157]:
MainRecommender.prepare_matrix(data_train).head()

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Необходимо ли сделать новые функции в классе тоже @staticmethod? 